In [1]:
import numpy as np
import pandas as pd

# make sure you have installed the Plotly package
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

# fill in your username and api_key before using Plotly
plotly.tools.set_credentials_file(username = '5005p', 
                                  api_key = 'njeG2Tu75B6pcDoUQQwL')

data = pd.read_csv('TestCenterInfo.csv')

In [6]:
# the change curve of utilization of each states 
trace = []
for i in data.state.unique():
    state_i = data[data.state == i]
    y = []
    for j in sorted(state_i.time.unique()):
        y.append(state_i[state_i.time == j].utilization.mean())
    trace_i = go.Scatter(
        x = np.array(range(len(state_i.time.unique()))),
        y = np.array(y),
        mode = "lines + markers",
        name = i)
    trace.append(trace_i)

py.iplot(trace, filename = 'change curve of utilization')

In [8]:
# distribution cure of utilization of a certain state in the first half and the second half of the year
state_i = data[data.state == 'florida']

first_half = np.array(state_i[state_i.month < 6].utilization)
second_half = np.array(state_i[state_i.month >= 6].utilization)

trace_first = go.Histogram(
    x = first_half,
    opacity = 0.75,
    name = 'first half year',
)
trace_second = go.Histogram(
    x = second_half,
    opacity = 0.55,
    name = 'second half year'
)

trace = [trace_first, trace_second]

layout = go.Layout(barmode = 'overlay')

fig = go.Figure(data = trace, layout = layout)

py.iplot(fig, filename = 'distribution curve of utilization')


In [9]:
# observe the utilization pattern in different seasons by using contour plot
x=['spring', 'summer', 'autumn' , 'winter']
y=[]
z=[]

for i in data.state.unique()[6:10]:
    y.append(i)

for i in y:
    state_i = data[data.state == i]
    state_i_spring = state_i[((state_i.month > 0) & (state_i.month <= 3))]
    z.append(state_i_spring.utilization.mean())
    
for i in y:
    state_i = data[data.state == i]
    state_i_spring = state_i[((state_i.month > 0) & (state_i.month <= 6))]
    z.append(state_i_spring.utilization.mean())

for i in y:
    state_i = data[data.state == i]
    state_i_spring = state_i[((state_i.month > 6) & (state_i.month <= 9))]
    z.append(state_i_spring.utilization.mean())

for i in y:
    state_i = data[data.state == i]
    state_i_spring = state_i[((state_i.month > 9) & (state_i.month <= 12))]
    z.append(state_i_spring.utilization.mean())

t= []
for i in range(4):
    t.append(z[i * 4:(i+1) * 4])

trace = [
    go.Contour(
        z = t,
        x = y,
        y = x
    )]

py.iplot(trace)

In [32]:
trace = []
for i in data.state.unique():
    a = np.random.randint(70, 200)
    b = np.random.randint(70, 200)
    c = np.random.randint(70, 200)
    r = []
    t = ['FEB', 'APR', 'JUN', 'SEP', 'OCT', 'DEC']
    state_i = data[data.state == i]
    r.append(state_i[state_i.month == 2].utilization.mean())
    r.append(state_i[state_i.month == 4].utilization.mean())
    r.append(state_i[state_i.month == 6].utilization.mean())
    r.append(state_i[state_i.month == 9].utilization.mean())
    r.append(state_i[state_i.month == 10].utilization.mean())
    r.append(state_i[state_i.month == 12].utilization.mean())
    
    trace_i = go.Area(
        r = r,
        t = t,
        name = i,
        marker = dict(color = 'rgb({}, {}, {})'.format(a, b, c)))
    trace.append(trace_i)

layout = go.Layout(
    title = 'low utilization detection',
    font = dict(
        size = 16
        ),
    legend = dict(
        font = dict(
            size = 16
            )
        ),
    radialaxis = dict(
        ticksuffix = '%'
        ),
    orientation = 270
    )
fig = go.Figure(data = trace, layout = layout)
py.iplot(fig, filename = 'low utilization detection')  

In [3]:
trace = []
j = 0
for i in data.state.unique()[:2]:
    state_i = data[data.state == i]
    if j == 0:
        fill = None
    else:
        fill = 'tonexty'
    y = []
    for k in sorted(state_i.month.unique()):
        y.append(state_i[state_i.month == k].utilization.mean())
    trace_i = go.Scatter(
        x = np.array(range(len(state_i.month.unique()))),
        y = np.array(y),
        mode = "lines",
        fill = fill,
        name = i,
        line = dict(
        color = 'rgb(143, 19, 131)',
    ))
        
    trace.append(trace_i)
    j += 1

py.iplot(trace, filename = 'delta utilization')

In [18]:
rawData = []
for i in data.state.unique():
    point = {}
    state_i = data[data.state == i]
    point['population'] = (state_i.capacity.mean() - data.capacity.min()) \
                        / (data.capacity.max() - data.capacity.min())
    point['expense'] = (state_i.expense.mean() - data.expense.min()) \
                        / (data.expense.max() - data.expense.min())
    point['utilization'] = (state_i.assigned.mean() - data.assigned.min()) \
                        / (data.assigned.max() - data.assigned.min())
    point['label'] = i
    rawData.append(point)


def makeAxis(title, tickangle): 
    return {
      'title': title,
      'titlefont': { 'size': 20 },
      'tickangle': tickangle,
      'tickfont': { 'size': 15 },
      'tickcolor': 'rgba(0,0,0,0)',
      'ticklen': 5,
      'showline': True,
      'showgrid': True
    }

trace = [{ 
    'type': 'scatterternary',
    'mode': 'markers',
    'a': [i for i in map(lambda x: x['population'], rawData)],
    'b': [i for i in map(lambda x: x['expense'], rawData)],
    'c': [i for i in map(lambda x: x['utilization'], rawData)],
    'text': [i for i in map(lambda x: x['label'], rawData)],
    'marker': {
        'symbol': 100,
        'color': '#DB7365',
        'size': 14,
        'line': { 'width': 2 }
    },
    }]

layout = {
    'ternary': {
        'sum': 100,
        'aaxis': makeAxis('population', 0),
        'baxis': makeAxis('<br>expense', 45),
        'caxis': makeAxis('<br>utilization', -45)
    },
    'annotations': [{
      'showarrow': False,
      'text': 'Ternary Plot',
        'x': 0.5,
        'y': 1.3,
        'font': { 'size': 15 }
    }]
}

fig = {'data': trace, 'layout': layout}
py.iplot(fig, validate=False)

In [29]:
trace = []
for i in data.state.unique()[0:10]:
    state_i = data[data.state == i]
    a = np.random.randint(70, 230)
    b = np.random.randint(70, 230)
    c = np.random.randint(70, 230)
    trace_i = go.Scatter(
        r = np.array(state_i.assigned),
        t = np.array(state_i.capacity),
        mode = 'markers',
        name = i,
        marker = dict(
            color = 'rgb({}, {}, {})'.format(a, b, c),
            size = 110,
            line = dict(
                color = 'white'
                ),
         opacity = 0.7
        )
    )
    trace.append(trace_i)

layout = go.Layout(
    title=' ',
    font = dict(
        size = 15
    ),
    plot_bgcolor = 'rgb(223, 223, 223)',
    angularaxis = dict(
        tickcolor = 'rgb(253,253,253)'
    )
)
fig = go.Figure(data = trace, layout = layout)
py.iplot(fig)

In [33]:
df = data2
df['text'] = df['name'] + '<br>Population'
limits = [(0, 500), (500, 2000) ,(2000, 4000), (4000, 9000), (9000, 13000)]
colors = []
for i in range(len(limits)):
    colors.append("rgb({}, {}, {})".format(np.random.randint(100,220), \
                                           np.random.randint(100,220), \
                                           np.random.randint(100,220)))
cities = []
scale = 100

for i in range(len(limits)):
    lim = limits[i]
    df_sub = df[lim[0]:lim[1]]
    city = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_sub['lon'],
        lat = df_sub['lat'],
        text = df_sub['text'],
        marker = dict(
            size = df_sub['pop'] / scale,
            color = colors[i],
            line = dict(width = 0.5, 
                        color = 'rgb(40,40,40)'
                       ),
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0], lim[1])
    )
    cities.append(city)

layout = dict(
        title = 'population',
        showlegend = True,
        geo = dict(
            scope = 'usa',
            projection = dict( 
                type = 'albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth = 1,
            countrywidth = 1,
            subunitcolor = "rgb(255, 255, 255)",
            countrycolor = "rgb(255, 255, 255)"
        ),
    )

fig = dict( data = cities, layout = layout )
py.iplot( fig, validate = False, filename = 'population' )